Mills College

CS 141-241, Machine Learning Fall 2018

Instructor: Jahan Ghofraniha

Final Project

Due date: 12/18/2018

Ian de Heer and Juliana de Heer

### Helpful Resources:


*   Data description: https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b
*   panyao's Solution: https://github.com/panyao/instacart-solution
  * panyao's EDA: https://github.com/panyao/instacart-solution/blob/master/exploration/Data%20Exploratory%20for%20Instacart%20Market%20Basket%20Analysis.ipynb
*   Phillipp's EDA: https://www.kaggle.com/philippsp/exploratory-analysis-instacart/notebook
*   Logistic Regression Solution: https://towardsdatascience.com/whats-in-your-customer-s-next-shopping-cart-73d64287ec53
*   Someone's approach to handling the raw dataset: https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/34727
*   Deep Learning Solution (3rd place): https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/38097
*   Data Engineering + Boosting Solution (2nd place): https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/38143
  * Interview: http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/
*   Relationships between csv files: https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/33128
*   Solutions summary: https://www.kaggle.com/c/instacart-market-basket-analysis/discussion/38130

In [1]:
import pandas as pd

aisles_data = pd.read_csv('aisles.csv')
print("Number of aisles: ", len(aisles_data.aisle_id.unique()))
aisles_data.head()

Number of aisles:  134


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [3]:
departments_data = pd.read_csv('departments.csv')
print("Number of departments: ", len(departments_data.department_id.unique()))
departments_data.head()

Number of departments:  21


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
orders_data = pd.read_csv('orders.csv')
print("Number of oders: ", len(orders_data.order_id.unique()))
print("Number of customers: ", len(orders_data.user_id.unique()))
orders_data.head()

Number of oders:  3421083
Number of customers:  206209


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [6]:
products_data = pd.read_csv('products.csv')
print("Number of products: ", len(products_data.product_id.unique()))
products_data.head()

Number of products:  49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
sample = pd.read_csv('sample_submission.csv')
sample.head()

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [7]:
op_prior = pd.read_csv('order_products__prior.csv')
print(op_prior.shape)
op_prior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [8]:
op_train = pd.read_csv('order_products__train.csv')
print(op_train.shape)
op_train.head()

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [9]:
df1 = pd.merge(op_prior,products_data, on="product_id")
df1.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


In [10]:
op_prior.shape

(32434489, 4)

In [11]:
df1.shape

(32434489, 7)

In [12]:
df2 = pd.merge(df1, orders_data, on='order_id')
print(df2.shape)
df2.head()

(32434489, 13)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,202279,prior,3,5,9,8.0
1,2,28985,2,1,Michigan Organic Kale,83,4,202279,prior,3,5,9,8.0
2,2,9327,3,0,Garlic Powder,104,13,202279,prior,3,5,9,8.0
3,2,45918,4,1,Coconut Butter,19,13,202279,prior,3,5,9,8.0
4,2,30035,5,0,Natural Sweetener,17,13,202279,prior,3,5,9,8.0


In [13]:
df3 = pd.merge(df2, departments_data, on='department_id')
print(df3.shape)
df3.head()

(32434489, 14)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,department
0,2,33120,1,1,Organic Egg Whites,86,16,202279,prior,3,5,9,8.0,dairy eggs
1,26,33120,5,0,Organic Egg Whites,86,16,153404,prior,2,0,16,7.0,dairy eggs
2,26,35951,1,0,Organic Unsweetened Almond Milk,91,16,153404,prior,2,0,16,7.0,dairy eggs
3,120,33120,13,0,Organic Egg Whites,86,16,23750,prior,11,6,8,10.0,dairy eggs
4,120,31323,7,0,Light Wisconsin String Cheese,21,16,23750,prior,11,6,8,10.0,dairy eggs


In [14]:
df4 = pd.merge(df3, aisles_data, on='aisle_id')
print(df4.shape)
df4.head()

(32434489, 15)


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,department,aisle
0,2,33120,1,1,Organic Egg Whites,86,16,202279,prior,3,5,9,8.0,dairy eggs,eggs
1,26,33120,5,0,Organic Egg Whites,86,16,153404,prior,2,0,16,7.0,dairy eggs,eggs
2,120,33120,13,0,Organic Egg Whites,86,16,23750,prior,11,6,8,10.0,dairy eggs,eggs
3,327,33120,5,1,Organic Egg Whites,86,16,58707,prior,21,6,9,8.0,dairy eggs,eggs
4,390,33120,28,1,Organic Egg Whites,86,16,166654,prior,48,0,12,9.0,dairy eggs,eggs


In [15]:
import numpy as np

# to make the output stable across runs
np.random.seed(42)

# To plot pretty figures
#import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [16]:
print(df4.shape)
df = df4[['eval_set', 'user_id', 'order_id', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'product_id', 'product_name', 'aisle_id', 'aisle', 'department_id', 'department', 'add_to_cart_order', 'reordered']]
print(df.shape)

df.head(30)

(32434489, 15)
(32434489, 15)


,eval_set,user_id,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,product_name,aisle_id,aisle,department_id,department,add_to_cart_order,reordered
0,prior,202279,2,3,5,9,8.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,1,1
1,prior,153404,26,2,0,16,7.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,5,0
2,prior,23750,120,11,6,8,10.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,13,0
3,prior,58707,327,21,6,9,8.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,5,1
4,prior,166654,390,48,0,12,9.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,28,1
5,prior,180135,537,15,2,8,3.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,2,1
6,prior,193223,582,6,2,19,10.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,7,1
7,prior,91030,608,11,3,21,12.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,5,1
8,prior,37804,623,63,3,12,3.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,1,1
9,prior,108932,689,16,1,13,3.0,33120,Organic Egg Whites,86,eggs,16,dairy eggs,4,1


In [17]:
# data = pd.DataFrame()
# print(df[0])
print(df.iloc[0])

# for index, row in df.iterrows():
#     print(row["user_id"], row["product_id"])
# for row in df.itertuples()[:3]:
#     print(row)
# for row in df.itertuples(index=True, name='Pandas')[:3]:
#     print getattr(row, "user_id"), getattr(row, "product_id")
#     if row[user_id]+

eval_set                               prior
user_id                               202279
order_id                                   2
order_number                               3
order_dow                                  5
order_hour_of_day                          9
days_since_prior_order                     8
product_id                             33120
product_name              Organic Egg Whites
aisle_id                                  86
aisle                                   eggs
department_id                             16
department                        dairy eggs
add_to_cart_order                          1
reordered                                  1
Name: 0, dtype: object


In [ ]:
# print(df.loc[:5])
(df.groupby(['eval_set', 'user_id', 'product_id']).sum()
 .loc[lambda df: df.user_id < 100])

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Class', axis=1)  
y = dataset['Class']  
